In [ ]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import MultiLabelBinarizer
from tensorflow.keras.callbacks import TensorBoard
import datetime
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [21]:
# Configuration TensorBoard
%load_ext tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)







The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
df = pd.read_csv('db/questions_db.csv')
print(df.head)
# df = df.sample(frac=0.1, random_state=42) pour réduire l'échantillon si nécessaire

In [ ]:
# Nettoie le texte
def clean_text(text):
    if isinstance(text, str):
        # Enleve les symboles HTML
        text = re.sub(r'<.*?>', '', text)
        # Enleve les caractères spéciaux
        text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
        # Converti en minuscule
        text = text.lower()
    return text

# transforme les tags sous forme de chaîne de caractères en liste
def process_tags(tags):
    if isinstance(tags, str):
        
        # Enlève les symboles '<>' entourant les tags et les divise en une liste
        return tags.strip('<>').split('><')
    else:
        return []

# Applique le traitement des tags à la colonne 'Tags' du DataFrame
df['Tags'] = df['Tags'].apply(process_tags)


# Applique clean_text pour nettoyer les colonnes 'Title' et 'Body'
df['Title'] = df['Title'].apply(clean_text)
df['Body'] = df['Body'].apply(clean_text)


print(df.head())

In [ ]:
# Vérifie les valeurs manquantes
def check_missing_values(df):
    missing_values = df.isnull().sum()
    print("Nombre de valeurs manquantes par colonne:")
    print(missing_values)
    if missing_values.any():
        print("\nColonnes avec valeurs manquantes:")
        print(missing_values[missing_values > 0])
    else:
        print("Aucune valeur manquante trouvée.")


check_missing_values(df)

In [ ]:
# Crée une liste des tags présents dans le DataFrame
all_tags = [tag for tags in df['Tags'] for tag in tags]

# Vérifie si all_tags est bien généré
if all_tags:
    # Si oui, affiche le nombre total de tags extraits
    print(f"Nombre total de tags extraits: {len(all_tags)}")
else:
    print("Erreur : Aucun tag n'a été extrait. Vérifiez la colonne 'Tags' dans votre DataFrame.")

# Compte la fréquence de chaque tag et sélectionne les 50 tags les plus fréquents
tag_counts = pd.Series(all_tags).value_counts().head(50)

# Les convertie en une liste
top_tags = list(tag_counts.index)

# Filtre les questions pour ne conserver que celles avec les top 50 tags
df['Filtered_Tags'] = df['Tags'].apply(lambda tags: [tag for tag in tags if tag in top_tags])

# Supprime les questions sans tags dans le top 50
df = df[df['Filtered_Tags'].apply(len) > 0]

print(top_tags)
plt.figure(figsize=(12, 18))
sns.barplot(x=tag_counts.values, y=tag_counts.index)
plt.title('Top 20 des Tags les plus fréquents')
plt.xlabel('Fréquence')
plt.ylabel('Tags')
plt.show()

In [ ]:
# vérifie la présence de balises HTML et de caractères spéciaux dans un texte
def check_html_special_chars(text):
    if isinstance(text, str):
        # Trouve toutes les balises HTML dans le texte
        html_tags = re.findall(r'<.*?>', text)
        # Trouve tous les caractères spéciaux (non alphanumériques et non espaces) 
        special_chars = re.findall(r'[^a-zA-Z0-9\s]', text)
        # Retourne le nombre total de balises HTML et de caractères spéciaux trouvés
        return len(html_tags) + len(special_chars)
    return 0


# Applique check_html_special_chars à la colonne 'Title'
df['Title_html_special'] = df['Title'].apply(check_html_special_chars)
# Applique check_html_special_chars à la colonne 'Body'

df['Body_html_special'] = df['Body'].apply(check_html_special_chars)

print("Occurrences de caractères HTML ou spéciaux restants dans les titres")
print(df['Title_html_special'].value_counts())
print("Occurrences de caractères HTML ou spéciaux restants dans les corps")
print(df['Body_html_special'].value_counts())

# Vérification des tags
print("Vérification des tags")
print(df['Tags'].head(10))

# Affiche le tableau complet sans limite
print(df)

In [ ]:
def add_features(df):
    # Longueur du titre
    df['Title_length'] = df['Title'].apply(
        lambda x: len(x.split()) if isinstance(x, str) else 0)
    # Nombre de mots dans le corps
    df['Body_word_count'] = df['Body'].apply(
        lambda x: len(x.split()) if isinstance(x, str) else 0)
    # Nombre de tags
    df['Tag_count'] = df['Tags'].apply(
        lambda x: len(x) if isinstance(x, list) else 0)
    return df


# Ajoute les features supplémentaires
df = add_features(df)
print(df.head(10))

In [ ]:
df.info()

In [ ]:
df = df.drop(columns=['FavoriteCount'])

## Analyse multivariée


In [ ]:
cols_to_analyze = ['Score', 'ViewCount', 'AnswerCount',
                   'Title_length', 'Body_word_count', 'Tag_count']
score_col = 'Score'
view_col = 'ViewCount'
favorite_col = 'FavoriteCount'

In [ ]:
def analyse_univariee_multivariee(df, cols):
    print("Analyse Univariée")
    for column in cols:
        if column in df.columns:
            plt.figure(figsize=(10, 6))
            sns.histplot(df[column], kde=True)
            plt.title(f'Distribution de {column}')
            plt.xlabel(column)
            plt.ylabel('Fréquence')
            plt.show()

    print("Analyse Multivariée")
    sns.pairplot(df[cols].select_dtypes(include=['int64', 'float64']))
    plt.show()


analyse_univariee_multivariee(df, cols_to_analyze)

In [ ]:
print(df.describe())

In [ ]:
def distribution_scores_vues_favoris(df, score_col, view_col, favorite_col):
    if score_col in df.columns:
        plt.figure(figsize=(10, 6))
        sns.histplot(df[score_col], bins=30, kde=True)
        plt.title(f'Distribution de {score_col}')
        plt.xlabel(score_col)
        plt.ylabel('Fréquence')
        plt.show()

    if view_col in df.columns:
        plt.figure(figsize=(10, 6))
        sns.histplot(df[view_col], bins=30, kde=True)
        plt.title(f'Distribution de {view_col}')
        plt.xlabel(view_col)
        plt.ylabel('Fréquence')
        plt.show()

    if favorite_col in df.columns and df[favorite_col].notnull().sum() > 0:
        plt.figure(figsize=(10, 6))
        sns.histplot(df[favorite_col], bins=30, kde=True)
        plt.title(f'Distribution de {favorite_col}')
        plt.xlabel(favorite_col)
        plt.ylabel('Fréquence')
        plt.show()
    else:
        print(f"La colonne {favorite_col} est vide ou n'existe pas.")

In [ ]:
def correlation_variables(df, cols):
    corr_matrix = df[cols].corr()
    plt.figure(figsize=(12, 8))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
    plt.title('Matrice de Corrélation')
    plt.show()


correlation_variables(df, cols_to_analyze)

### Tokenization + stopwords + lemmatization

In [ ]:



# Nettoie le texte des mots vides
def treatments_text(text):
    # Tokenization
    tokens = word_tokenize(text)

    # Suppression des stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.lower() not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return ' '.join(tokens)

# Colonnes Title et Body


def clean_dataframe(df):
    df['Cleaned_Title'] = df['Title'].apply(treatments_text)
    df['Cleaned_Body'] = df['Body'].apply(treatments_text)
    return df


df = clean_dataframe(df)

# Verification
print(df[['Title', 'Cleaned_Title', 'Body', 'Cleaned_Body']].head())

In [ ]:

def transform_text_to_bow(df):
    # Bag of Words : compte les mots et les represente en format numérique 
    vectorizer = CountVectorizer(max_df=0.95, min_df=2)
    X = vectorizer.fit_transform(df['Cleaned_Body'])

    # Réduction de dimension avec TruncatedSVD : réduit la dimensionnalité des vecteurs BoW
    svd = TruncatedSVD(n_components=100)
    X_reduced = svd.fit_transform(X)

    # Transformation des tags : convertit les tags en une forme utilisable pour l'apprentissage supervisé
    mlb = MultiLabelBinarizer(classes=top_tags)
    y = mlb.fit_transform(df['Filtered_Tags'])


    return X_reduced, y, vectorizer, svd, mlb

In [ ]:
X_reduced, y, vectorizer, svd, mlb = transform_text_to_bow(df)

# Vérification que X_reduced et y ne sont pas None
print("Shape of X_reduced:", X_reduced.shape if X_reduced is not None else "X_reduced is None")
print("Shape of y:", y.shape if y is not None else "y is None")


In [ ]:
# Sauvegare avec pickle de quelques valeurs a travailler dans un autre notebook

import pickle

# Sauvegarde des objets
with open('X_reduced.pkl', 'wb') as f:
    pickle.dump(X_reduced, f)

with open('y.pkl', 'wb') as f:
    pickle.dump(y, f)

with open('vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

with open('svd.pkl', 'wb') as f:
    pickle.dump(svd, f)

with open('top_tags.pkl', 'wb') as f:
    pickle.dump(top_tags, f)

# Affichage de top_tags après la sauvegarde
print(top_tags)
